In [1]:
import pandas as pd

from pandarallel import pandarallel

from freeholdforecast.tasks.etl_ml_task import ETL_ML_Task

pd.options.display.max_columns = 100

task = ETL_ML_Task()


2022-09-04 20:25:06 INFO ETL_ML_Task Initialized task for ohio-hamilton with run date 2016-09-01


In [2]:
task._get_df_raw_encoded()
task.df_raw_encoded


2022-09-04 20:25:06 INFO ETL_ML_Task Retrieving and encoding data
2022-09-04 20:25:06 INFO ETL_ML_Task Loading existing raw data
2022-09-04 20:25:08 INFO ETL_ML_Task Total parcels: 280902
2022-09-04 20:25:08 INFO ETL_ML_Task Total sales: 497550
2022-09-04 20:25:08 INFO ETL_ML_Task Loading existing encoded data


,Book,Plat,Parcel,ParcelID,Tax District,Owner Name 1,Owner Name 2,Land Value,Building Value,Property Class,House #,Street Name,Street Suffix,Zip Code,Month of Sale,Day of Sale,Year of Sale,# of Parcels Sold,Sale Price,Valid Sale,Conveyance #,Deed Type,Appraisal Area,PriorOwner,PropertyNumber,Parid,last_sale_date
0,661.0,53.0,385.0,0.0,49.0,130710.0,36638.0,2666.0,38503.0,104.0,12240.0,13095.0,21.0,25526.0,0.0,3.0,0.0,2.0,1893.0,3.0,304881.0,44.0,101.0,205152.0,174522.0,128253.0,1998-01-01
1,648.0,2.0,134.0,0.0,31.0,167209.0,1.0,6121.0,2720.0,46.0,6986.0,8889.0,52.0,25526.0,0.0,29.0,0.0,24.0,8570.0,3.0,304726.0,24.0,101.0,205152.0,174522.0,101161.0,1998-01-01
2,683.0,60.0,268.0,0.0,58.0,150050.0,30862.0,4334.0,38422.0,104.0,2268.0,9175.0,21.0,25526.0,0.0,29.0,0.0,2.0,1998.0,3.0,304738.0,44.0,101.0,205152.0,174522.0,158588.0,1998-01-01
3,683.0,60.0,218.0,0.0,58.0,289633.0,42886.0,5138.0,736.0,104.0,20557.0,4510.0,19.0,25526.0,0.0,26.0,0.0,2.0,2823.0,3.0,304489.0,44.0,101.0,205152.0,174522.0,158564.0,1998-01-01
4,661.0,23.0,329.0,0.0,49.0,39291.0,1.0,6637.0,40802.0,104.0,11701.0,9088.0,19.0,25526.0,0.0,3.0,0.0,2.0,1637.0,3.0,304875.0,50.0,101.0,205152.0,174522.0,125059.0,1998-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497545,704.0,217.0,188.0,0.0,42.0,196975.0,57262.0,9525.0,39525.0,104.0,32102.0,18560.0,21.0,12221.0,6.0,26.0,24.0,2.0,5728.0,3.0,191315.0,50.0,87.0,204044.0,130786.0,212461.0,2022-07-01
497546,657.0,154.0,209.0,0.0,19.0,99536.0,57262.0,19743.0,17872.0,104.0,27334.0,2277.0,21.0,0.0,6.0,28.0,24.0,2.0,17551.0,3.0,191643.0,50.0,2.0,132631.0,66420.0,111939.0,2022-07-01
497547,683.0,62.0,132.0,0.0,58.0,173777.0,57262.0,7519.0,37434.0,104.0,31302.0,9818.0,21.0,16250.0,6.0,19.0,24.0,2.0,25977.0,3.0,193368.0,50.0,22.0,203468.0,97054.0,158885.0,2022-07-01
497548,685.0,62.0,288.0,0.0,50.0,18851.0,57262.0,6022.0,34136.0,111.0,13387.0,6144.0,21.0,0.0,6.0,21.0,24.0,2.0,3699.0,3.0,193541.0,50.0,36.0,41340.0,101181.0,168678.0,2022-07-01


In [3]:
task._get_df_prepared()
task.df_prepared


2022-09-04 20:25:08 INFO ETL_ML_Task Preparing data


2022-09-04 20:32:20 INFO ETL_ML_Task Saving prepared data
2022-09-04 20:32:38 INFO ETL_ML_Task Splitting data
2022-09-04 20:32:38 INFO ETL_ML_Task Train dates: 2006-08-01 to 2016-08-01
2022-09-04 20:32:38 INFO ETL_ML_Task Test dates: 2016-09-01 to 2018-09-01


,transfer_in_6_months,transfer_in_12_months,transfer_in_24_months,date,year,month,dates_since_last_sale,Book,Plat,Parcel,ParcelID,Tax District,Owner Name 1,Owner Name 2,Land Value,Building Value,Property Class,House #,Street Name,Street Suffix,Zip Code,Month of Sale,Day of Sale,Year of Sale,# of Parcels Sold,Sale Price,Valid Sale,Conveyance #,Deed Type,Appraisal Area,PriorOwner,PropertyNumber,Parid,last_sale_date
0,0,0,0,2004-06-01,2004,6,0,563.0,39.0,68.0,0.0,31.0,28004.0,2.0,12390.0,0.0,96.0,8091.0,15191.0,54.0,25526.0,5.0,0.0,6.0,2.0,19377.0,1.0,22550.0,31.0,101.0,205152.0,174522.0,78697.0,2004-06-01
1,0,0,0,2004-07-01,2004,7,1,563.0,39.0,68.0,0.0,31.0,28004.0,2.0,12390.0,0.0,96.0,8091.0,15191.0,54.0,25526.0,5.0,0.0,6.0,2.0,19377.0,1.0,22550.0,31.0,101.0,205152.0,174522.0,78697.0,2004-06-01
2,0,0,0,2004-08-01,2004,8,2,563.0,39.0,68.0,0.0,31.0,28004.0,2.0,12390.0,0.0,96.0,8091.0,15191.0,54.0,25526.0,5.0,0.0,6.0,2.0,19377.0,1.0,22550.0,31.0,101.0,205152.0,174522.0,78697.0,2004-06-01
3,0,0,0,2004-09-01,2004,9,3,563.0,39.0,68.0,0.0,31.0,28004.0,2.0,12390.0,0.0,96.0,8091.0,15191.0,54.0,25526.0,5.0,0.0,6.0,2.0,19377.0,1.0,22550.0,31.0,101.0,205152.0,174522.0,78697.0,2004-06-01
4,0,0,0,2004-10-01,2004,10,4,563.0,39.0,68.0,0.0,31.0,28004.0,2.0,12390.0,0.0,96.0,8091.0,15191.0,54.0,25526.0,5.0,0.0,6.0,2.0,19377.0,1.0,22550.0,31.0,101.0,205152.0,174522.0,78697.0,2004-06-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12752394,0,0,0,2022-04-01,2022,4,255,683.0,9.0,201.0,0.0,129.0,170346.0,1.0,4842.0,33538.0,104.0,18714.0,11508.0,19.0,25526.0,0.0,7.0,3.0,2.0,26264.0,3.0,250923.0,50.0,101.0,205152.0,174522.0,154930.0,2001-01-01
12752395,0,0,0,2022-05-01,2022,5,256,683.0,9.0,201.0,0.0,129.0,170346.0,1.0,4842.0,33538.0,104.0,18714.0,11508.0,19.0,25526.0,0.0,7.0,3.0,2.0,26264.0,3.0,250923.0,50.0,101.0,205152.0,174522.0,154930.0,2001-01-01
12752396,0,0,0,2022-06-01,2022,6,257,683.0,9.0,201.0,0.0,129.0,170346.0,1.0,4842.0,33538.0,104.0,18714.0,11508.0,19.0,25526.0,0.0,7.0,3.0,2.0,26264.0,3.0,250923.0,50.0,101.0,205152.0,174522.0,154930.0,2001-01-01
12752397,0,0,0,2022-07-01,2022,7,258,683.0,9.0,201.0,0.0,129.0,170346.0,1.0,4842.0,33538.0,104.0,18714.0,11508.0,19.0,25526.0,0.0,7.0,3.0,2.0,26264.0,3.0,250923.0,50.0,101.0,205152.0,174522.0,154930.0,2001-01-01


In [4]:
task._train_model("transfer_in_6_months")


2022-09-04 20:32:40 INFO ETL_ML_Task Training model for transfer_in_6_months
2022-09-04 20:32:44 INFO ETL_ML_Task Train labels: 136034/6072331 (2.24%)
2022-09-04 20:32:44 INFO ETL_ML_Task Train res labels: 136034/1496374 (9.09%)
2022-09-04 20:32:44 INFO ETL_ML_Task Test labels: 40084/1549540 (2.59%)
2022-09-04 20:32:44 INFO ETL_ML_Task Fit minutes: 60
2022-09-04 20:32:44 INFO ETL_ML_Task Per model fit minutes: 30
2022-09-04 20:32:44 INFO ETL_ML_Task Per model memory limit (MB): 2605
2022-09-04 20:32:44 INFO ETL_ML_Task Fitting model
[WARNING] [2022-09-04 20:32:47,744:Client-AutoML(1):bb34e497-2c90-11ed-aaa2-0242ac110002] Dataset too large for memory limit 2605MB, reducing the precision from float64 to <class 'numpy.float32'>
[WARNING] [2022-09-04 20:32:47,815:Client-AutoML(1):bb34e497-2c90-11ed-aaa2-0242ac110002] Dataset too large for memory limit 2605MB, reducing number of samples from 1496374 to 1177388.
[WARNING] [2022-09-04 20:32:50,074:Client-AutoML(1):bb34e497-2c90-11ed-aaa2-0242

/home/vscode/.local/share/virtualenvs/freehold-forecast-zrIN4e5_/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


2022-09-04 21:35:07 INFO ETL_ML_Task auto-sklearn results:
  Dataset name: bb34e497-2c90-11ed-aaa2-0242ac110002
  Metric: average_precision
  Best validation score: 0.934368
  Number of target algorithm runs: 76
  Number of successful target algorithm runs: 57
  Number of crashed target algorithm runs: 3
  Number of target algorithms that exceeded the time limit: 11
  Number of target algorithms that exceeded the memory limit: 5

2022-09-04 21:38:45 INFO ETL_ML_Task Pred labels: 8638/1549540 (0.56%)
2022-09-04 21:38:47 INFO ETL_ML_Task Precision: 0.47
2022-09-04 21:38:47 INFO ETL_ML_Task ROC AUC: 0.55
2022-09-04 21:38:48 INFO ETL_ML_Task Classification report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99   1509456
           1       0.47      0.10      0.17     40084

    accuracy                           0.97   1549540
   macro avg       0.72      0.55      0.58   1549540
weighted avg       0.96      0.97      0.97   1549540

2022

In [5]:
task._train_model("transfer_in_12_months")


2022-09-04 21:39:02 INFO ETL_ML_Task Training model for transfer_in_12_months
2022-09-04 21:39:06 INFO ETL_ML_Task Train labels: 252972/6072331 (4.17%)
2022-09-04 21:39:06 INFO ETL_ML_Task Train res labels: 252972/2782692 (9.09%)
2022-09-04 21:39:06 INFO ETL_ML_Task Test labels: 76654/1549540 (4.95%)
2022-09-04 21:39:06 INFO ETL_ML_Task Fit minutes: 60
2022-09-04 21:39:06 INFO ETL_ML_Task Per model fit minutes: 30
2022-09-04 21:39:06 INFO ETL_ML_Task Per model memory limit (MB): 2605
2022-09-04 21:39:06 INFO ETL_ML_Task Fitting model
[WARNING] [2022-09-04 21:39:07,858:Client-AutoML(1):00e0b33c-2c9a-11ed-aaa2-0242ac110002] Dataset too large for memory limit 2605MB, reducing the precision from float64 to <class 'numpy.float32'>
[WARNING] [2022-09-04 21:39:07,961:Client-AutoML(1):00e0b33c-2c9a-11ed-aaa2-0242ac110002] Dataset too large for memory limit 2605MB, reducing number of samples from 2782692 to 1177388.
[WARNING] [2022-09-04 21:39:10,530:Client-AutoML(1):00e0b33c-2c9a-11ed-aaa2-024

In [6]:
task._train_model("transfer_in_24_months")


2022-09-04 22:42:47 INFO ETL_ML_Task Training model for transfer_in_24_months
2022-09-04 22:42:53 INFO ETL_ML_Task Train labels: 470873/6072331 (7.75%)
2022-09-04 22:42:53 INFO ETL_ML_Task Train res labels: 470873/5179603 (9.09%)
2022-09-04 22:42:53 INFO ETL_ML_Task Test labels: 143405/1549540 (9.25%)
2022-09-04 22:42:53 INFO ETL_ML_Task Fit minutes: 60
2022-09-04 22:42:53 INFO ETL_ML_Task Per model fit minutes: 30
2022-09-04 22:42:53 INFO ETL_ML_Task Per model memory limit (MB): 2605
2022-09-04 22:42:53 INFO ETL_ML_Task Fitting model
[WARNING] [2022-09-04 22:42:56,284:Client-AutoML(1):e9f0d3c0-2ca2-11ed-aaa2-0242ac110002] Dataset too large for memory limit 2605MB, reducing the precision from float64 to <class 'numpy.float32'>
[WARNING] [2022-09-04 22:42:56,510:Client-AutoML(1):e9f0d3c0-2ca2-11ed-aaa2-0242ac110002] Dataset too large for memory limit 2605MB, reducing number of samples from 5179603 to 1177388.
[WARNING] [2022-09-04 22:43:00,200:Client-AutoML(1):e9f0d3c0-2ca2-11ed-aaa2-02